In [50]:
from qdrant_client import QdrantClient, models

In [51]:
client = QdrantClient(url="http://localhost:6333")

In [52]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=4, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6],
        payloads=[
            {"color": "red", "city": "London", "age": 24},
            {"color": "green", "city": "Moscow", "age": 34},
            {"color": "blue", "city": "Moscow", "age": 13},
            {"color": "green", "city": "London", "age": 56},
            {"color": "red", "city": "Berlin", "age": 88},
            {"color": "blue", "city": "London", "age": 8},
        ],
        vectors=[
            [0.1, 0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6, 0.6],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [53]:
# Retrieves points that are closer to stored positive examples and further from negative examples.
client.recommend(
    collection_name="collection0",
    positive=[1, 2],
    negative=[
        3,  # id(必须存在)
        [0.2, 0.3, 0.4, 0.5]  # 向量
    ],
    # Average vector strategy
    # The way to produce the searching vector is by first averaging all the positive and negative examples separately, and then combining them into a single vector using the following formula:
    # `avg_positive + avg_positive - avg_negative`
    # In the case of not having any negative examples, the search vector will simply be equal to avg_positive.
    strategy=models.RecommendStrategy.AVERAGE_VECTOR,  # potive必须存在
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(
                    value="London",
                ),
            )
        ]
    ),  # 参数含义与serach等价
    limit=1,  # 参数含义与serach等价
    with_payload=True,  # 参数含义与search等价
    with_vectors=True,  # 参考含义与search等价
)

[ScoredPoint(id=4, version=0, score=-0.04, payload={'age': 56, 'city': 'London', 'color': 'green'}, vector=[0.4, 0.4, 0.4, 0.4], shard_key=None, order_value=None)]

In [ ]:
client.recommend(
    collection_name="collection0",
    positive=[1, 2],
    negative=[3,  [0.2, 0.3, 0.4, 0.5]],
    # Best score strategy
    # It is based on the idea that the best way to find similar vectors is to find the ones that are closer to a positive example, while avoiding the ones that are closer to a negative one. 
    # The way it works is that each candidate is measured against every example, then we select the best positive and best negative scores. The final score is chosen with this step formula:
    # ```rust
    # let score = if best_positive_score > best_negative_score {
    #     best_positive_score;
    # } else {
    #     -(best_negative_score * best_negative_score);
    # };
    # ```
    # A beneficial side-effect of best_score strategy is that you can use it with only negative examples. This will allow you to find the most dissimilar vectors to the ones you provide. This can be useful for finding outliers in your data, or for finding the most dissimilar vectors to a given one.
    strategy=models.RecommendStrategy.BEST_SCORE,  # potive可以不存在
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(
                    value="London",
                ),
            )
        ]
    ), 
    with_payload=True, 
    with_vectors=True,
)